# INTRO
L'obiettivo di questo esercizio è quello di costruire un modello di machine learning in grado di distinguere i clienti frodatori dagli altri.

# DATASET
Il dataset è composto sia da colonne in chiaro che da colonne anonomizzate ed è una fotografia mensile dei clienti.
Di seguito la spiegazione delle colonne:

- *data_rif*: data di fine mese di riferimento
- *userid*: id del cliente
- *age*: età 
- *profession*: professione
- *region*: regione di residenza
- *account_balance*: saldo del conto a fine mese
- *num_trx_cd*: numero di transazioni con carta di debito eseguite nel mese
- *num_trx_cc*: numero di transazioni con carta di credito eseguite nel mese
- *num_trx_cp*: numero di transazioni con carta prepagata eseguite nel mese
- *num_mov_conto*: numero di movimenti di conto corrente nel mese
- *sum_mov_conto_pos*: somma degli importi dei movimenti in ingresso di conto corrente nel mese
- *sum_mov_conto_neg*: somma degli importi dei movimenti in uscita di conto corrente nel mese
- *num_prodotti*: numero di prodotti posseduti dal cliente
- *f2*, *f3*, *f4*, *f5*, *f6*, *f7*: features comportamentali anonimizzate
- *TARGET*: variabile target che indica se il cliente ha commesso una frode nei mesi successivi


# REQUISITI AGGIUNTIVI
- Possibile utilizzare qualunque libreria di ML purchè venga fornita la versione in modo da poter replicare i risultati
- Si preferiscono modelli "white-box" o spiegabili

# lettura dati

In [1]:
import pandas as pd
import numpy as np

In [2]:
DTYPES = {
    'data_rif': str,
    'userid': np.int64,
    'age':  np.int64,
    'profession': str,
    'region': str,
    'account_balance': np.float64,
    'num_trx_cd': np.float64,
    'num_trx_cc': np.float64,
    'num_trx_cp': np.float64,
    'num_mov_conto': np.int32,
    'sum_mov_conto_pos': np.int64,
    'sum_mov_conto_neg': np.int64,
    'num_prodotti': np.int64,
    'f2': np.int64,
    'f3': np.int64,
    'f4': np.int64,
    'f5': np.int64,
    'f6': np.float64,
    'f7': np.float64,
    'TARGET': np.int64
}

In [3]:
df = pd.read_csv("../data/frauds_dataset.csv", sep="~", dtype=DTYPES)

In [4]:
df.head()

,data_rif,userid,age,profession,region,account_balance,num_trx_cd,num_trx_cc,num_trx_cp,num_mov_conto,sum_mov_conto_pos,sum_mov_conto_neg,num_prodotti,f2,f3,f4,f5,f6,f7,TARGET
0,2022-07-31,1000510,23,Lavoratore autonomo,TOSCANA,65627.799269,0.0,0.0,0.0,10,3590,-370,2,88,60,8,20,21.141686,0.268369,0
1,2022-07-31,1001511,55,Lavoratore dipendente,BASILICATA,39335.109963,7.0,0.0,0.0,0,0,0,5,97,63,11,82,38.169452,0.672864,1
2,2022-07-31,1001726,23,Lavoratore autonomo,PUGLIA,-37466.828926,148.0,0.0,0.0,2,636,-294,10,90,49,31,71,38.602380,0.126743,0
3,2022-07-31,1002418,43,Studente,VALLE AOSTA,13864.880197,215.0,0.0,0.0,8,1064,-1640,3,99,66,52,57,31.505413,2.081956,1
4,2022-07-31,1002646,26,Studente,LOMBARDIA,-32625.910843,38.0,56.0,6.0,0,0,0,1,115,56,44,28,36.882651,0.210746,0


# analisi dati

# training del modello

# valutazione delle performance 